# NLP wiht LSTM TensorFlow Youtube Tutorial 

Video: https://www.youtube.com/watch?v=ZMudJXhsUpY&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=6

Code: https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%204%20-%20Lesson%202%20-%20Notebook.ipynb#scrollTo=w9vH8Y59ajYL

In [114]:
traning_new_model = True                # Soll ein neues Modell traniert werden ?
copie_js_model_to_react = True          # Soll das js Modell direkt rüber kopiert werden ?

## Setup 

In [115]:
import tensorflow as tf
import keras 

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import numpy as np 
import os

#os.environ["TF_USE_LEGACY_KERAS"] = "1" # https://github.com/tensorflow/tfjs/issues/8328#issuecomment-2212490809

! echo $TF_USE_LEGACY_KERAS
! python3 --version
print(tf.__version__)
print(keras.__version__)


1


Python 3.10.15
2.15.1
2.15.0


In [ ]:
# Modellname ohne führenden Schrägstrich
model_name = "lstm"  # ohne führendes "/"

text_file = os.path.abspath("../../example_text.txt")
model_file = os.path.join(os.path.abspath("./models"), model_name + ".h5")
js_model_dir = os.path.join(os.path.abspath("./models"), model_name + "_js")
react_model_path = os.path.join(os.path.abspath("/workspace/webpage/silvan-webpage/public/models"))

print(js_model_dir)
print(react_model_path)

/workspace/algorithem/next_word_prediciton/rnns/models/lstm_js
/workspace/webpage/silvan-webpage/public/models


## Data

In [117]:
# Datensatz laden
data = open(text_file, 'rb').read().decode(encoding='utf-8')

print(f"Inhalte des Datensatzes: \n\n{data[:250]}")

Inhalte des Datensatzes: 

Wie geht es dir?
Mir geht's gut, danke.
Was machst du heute?
Hast du gut geschlafen?
Wo wohnst du?
Ich wohne in Berlin.
Wie spät ist es?
Es ist schon ziemlich spät.
Hast du heute Zeit?
Was hast du am Wochenende vor?
Kannst du mir bitte helf


In [118]:
# Corpus erstellen
tokenizer = Tokenizer()
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(f"Der Tokenizer kennt {total_words} Wörter")
print(f"Die Wörter werden folgenden Indexe zugenortnet: {tokenizer.word_index}")


Der Tokenizer kennt 200 Wörter
Die Wörter werden folgenden Indexe zugenortnet: {'\r': 1, 'du': 2, 'ich': 3, 'das': 4, 'hast': 5, 'habe': 6, 'was': 7, 'wie': 8, 'es': 9, 'ist': 10, 'heute': 11, 'schon': 12, 'kannst': 13, 'möchtest': 14, 'hier': 15, 'wann': 16, 'nicht': 17, 'lange': 18, 'kann': 19, 'wo': 20, 'in': 21, 'zeit': 22, 'zu': 23, 'noch': 24, 'zum': 25, 'der': 26, 'bin': 27, 'dir': 28, 'mir': 29, 'gut': 30, 'bitte': 31, 'etwas': 32, 'gehen': 33, 'muss': 34, 'war': 35, 'machst': 36, 'spät': 37, 'ziemlich': 38, 'wochenende': 39, 'frei': 40, 'wir': 41, 'uns': 42, 'mich': 43, 'schön': 44, 'den': 45, 'film': 46, 'viel': 47, 'tun': 48, 'mit': 49, 'keine': 50, 'einen': 51, 'dauert': 52, 'finde': 53, 'die': 54, 'nächste': 55, 'gibt': 56, 'komme': 57, 'bleibst': 58, 'gleich': 59, 'an': 60, 'meinen': 61, 'vergessen': 62, 'dein': 63, 'morgen': 64, 'bus': 65, 'weit': 66, 'bis': 67, 'eine': 68, 'geht': 69, "geht's": 70, 'danke': 71, 'geschlafen': 72, 'wohnst': 73, 'wohne': 74, 'berlin': 75, 

In [119]:
# Datensatu generieren
input_sequences = []

# Wandelt jede Zeile in eine Liste von Wortindizes 
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]

	# n-Gramm-Sequenz generieren
	for i in range(1, len(token_list)):

		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# Alle Sequences auf gleiche Laenge bringen 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Aufteilen in Prädiktoren und Labels
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

## Modell

In [ ]:
# Modell Struktur 

from tensorflow.keras.models import load_model

def create_model():
    model = Sequential()
    model.add(Embedding(input_dim=total_words,                  # Anzahl aller Wörter
                        output_dim=100,                         # Umwandlung des Wortes (des Integers) in den einen n-dimensionanlen Vektor 
                        input_length=(max_sequence_len - 1)))   # Länge der Eingaben
    model.add(Bidirectional(LSTM(150)))
    model.add(Dense(total_words, activation='softmax'))
    
    # Optimizer und Modell-Compile
    adam = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

# Prüfen, ob das Modell bereits existiert
if os.path.exists(model_file) and traning_new_model == False:
    print("Modell gefunden. Lade bestehendes Modell...")
    model = load_model(model_file)  
    
else:
    if traning_new_model ==True:
        print("Neuse Model soll traniert werden")
    else: 
        print("Altes Modell nicht gefunden")
    
    print("Startet Training \n\n")
    model = create_model()  # Neues Modell erstellen
    
    # Modell trainieren
    history = model.fit(xs, ys, epochs=100, verbose=1)
    
    # Modell speichern
    model.save(model_file)
    print(f"Modell gespeichert unter: {model_file}")


Neuse Model soll traniert werden
Startet Training 




Epoch 1/100
15/15 [==============================] - 5s 39ms/step - loss: 4.8793 - accuracy: 0.1979
Epoch 2/100
15/15 [==============================] - 1s 38ms/step - loss: 4.1645 - accuracy: 0.2702
Epoch 3/100
15/15 [==============================] - 1s 37ms/step - loss: 3.7144 - accuracy: 0.2957
Epoch 4/100
15/15 [==============================] - 1s 35ms/step - loss: 3.2407 - accuracy: 0.3404
Epoch 5/100
15/15 [==============================] - 1s 38ms/step - loss: 2.8545 - accuracy: 0.3681
Epoch 6/100
15/15 [==============================] - 0s 26ms/step - loss: 2.3944 - accuracy: 0.4064
Epoch 7/100
15/15 [==============================] - 1s 34ms/step - loss: 2.0164 - accuracy: 0.4447
Epoch 8/100
15/15 [==============================] - 1s 38ms/step - loss: 1.5327 - accuracy: 0.5809
Epoch 9/100
15/15 [==============================] - 1s 41ms/step - loss: 1.2025 - accuracy: 0.6596
Epoch 10/100
15/15 [==============================] - 0s 30ms/step - loss: 0.9533 - accuracy: 0.6851

In [121]:
# Zugriff auf die Dimensionen der Embedding-Schicht
embedding_layer = model.layers[0]  # Die erste Schicht ist die Embedding-Schicht
embedding_input_dim = embedding_layer.input_dim  # Eingabedimensionen (total_words)
embedding_output_dim = embedding_layer.output_dim  # Ausgabedimensionen (Embedding Dimension)
embedding_weights = embedding_layer.get_weights()  # Zugriff auf die Gewichte der Embedding-Schicht

print(f"Embedding Eingabedimensionen: {embedding_input_dim}")
print(f"Embedding Ausgabedimensionen: {embedding_output_dim}")
print(f"Embedding Gewichtedimensionen: {embedding_weights[0].shape}")  # Die Dimensionen der Gewichtsmatrix

Embedding Eingabedimensionen: 200
Embedding Ausgabedimensionen: 100
Embedding Gewichtedimensionen: (200, 100)


## Verwendung 

### Text generieren

In [122]:
seed_text = "Hallo ich bin"
next_words = 1
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 1s 790ms/step
Hallo ich bin gleich


### Nextes Word vorhesagen  

In [123]:
import numpy as np

seed_text = "Wie"
top_k = 10  # Anzahl der Top-Wörter

# Tokenisierung und Padding des Eingabetexts
token_list = tokenizer.texts_to_sequences([seed_text])[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

# Vorhersagen berechnen
predictions = model.predict(token_list, verbose=0)[0]  # Softmax-Ausgabe

# Top-10 wahrscheinlichste Wörter finden
top_indices = np.argsort(predictions)[-top_k:][::-1]  #Sortiere absteigend

# Wörter und Wahrscheinlichkeiten sammeln
top_words = [(word, predictions[index]) for word, index in tokenizer.word_index.items() if index in top_indices]

# Sortiere die Wörter nach Wahrscheinlichkeit in absteigender Reihenfolge
top_words = sorted(top_words, key=lambda x: x[1], reverse=True)

# Ausgabe der Top-10 Wörter und Wahrscheinlichkeiten
print(f"Top {top_k} nächste Wörter für '{seed_text}':")
for word, prob in top_words:
    print(f"{word}: {(prob * 100):.4f} %")


Top 10 nächste Wörter für 'Wie':
lange: 34.8497 %
war: 21.2845 %
groß: 9.6902 %
geht: 8.9980 %
spät: 8.3389 %
komme: 8.2226 %
weit: 6.3798 %
ist: 0.1222 %
dein: 0.0559 %
ziemlich: 0.0552 %


## Konvertierung 

In [124]:
if traning_new_model:
    print("Konveriert neues Modell \n\n")
    ! tensorflowjs_converter --input_format=keras {model_file} {js_model_dir}
else:
    print("Konvertiertes Model bereits vorhanden")


Konveriert neues Modell 


/bin/bash: line 1: tensorflowjs_converter: command not found


In [125]:
if copie_js_model_to_react:
    print("Kopiere JS-Model in React-Model")
    ! cp -r {js_model_dir} {react_model_path}
else:
    print("JS-Model bereits im React-Model vorhanden")

Kopiere JS-Model in React-Model
